# Final Project
### Import precomp libs

In [7]:
import spacy
nlp = spacy.load('en') # this should take some time like 10s to load
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [ ]:
#import the spacy english model
# "python -m spacy download en"   to download english models

import pickle 
import numpy as np
import scipy as sp
import pandas as pd
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import coo_matrix, hstack

In [ ]:
# PROJECT structure: data directory with the two csvs of train,test and the ipython notbook in its own directory
# loading the data
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
print train.shape, test.shape
train[379205]

In [ ]:
from nltk.stem.porter import PorterStemmer
def stem 
    porter = PorterStemmer()
    for w in l:
        porter.stem(w)
stem('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

In [11]:
def parse(sentence):
    bow = []
    sentence = sentence.decode('utf-8')
    p = nlp(sentence)
    for word in p:
        if word.is_stop: continue
        bow.append(word.lemma_)
    return Counter(bow)
        
parse('What are the stories of Kohinoor (Koh-i-Noor) Diamond? men walked to the stores to buy shirts')

Counter({u'(': 1,
         u')': 1,
         u'-': 2,
         u'?': 1,
         u'buy': 1,
         u'diamond': 1,
         u'koh': 1,
         u'kohinoor': 1,
         u'man': 1,
         u'noor': 1,
         u'shirt': 1,
         u'store': 1,
         u'story': 1,
         u'walk': 1})

In [12]:
def print_vecs(corpus):
    vect_list = []
    results = []
    n = len(train)
    for i in range(n): 
        if i > 10: return 
        f = train.loc[i]
        q1, q2 = f['question1'], f['question2']
        first = parse(q1)
        first.subtract(parse(q2))
        print first
print_vecs(train)

Counter({u'india': 1, u'invest': 0, u'step': 0, u'?': 0, u'share': 0, u'guide': 0, u'market': 0})
Counter({u'story': 1, u'koh': 0, u'diamond': 0, u')': 0, u'(': 0, u'-': 0, u'kohinoor': 0, u'noor': 0, u'?': 0, u'government': -1, u'indian': -1, u'happen': -1, u'steal': -1})
Counter({u'connection': 1, u'vpn': 1, u'increase': 0, u'internet': 0, u'speed': 0, u'?': 0, u'hack': -1, u'dns': -1})
Counter({u'lonely': 1, u'solve': 1, u'mentally': 1, u'?': 1, u'divide': -1, u']': -1, u'math]23^{24}[/math': -1, u'24,23': -1, u'[': -1, u'remainder': -1, u'find': -1})
Counter({u',': 2, u'di': 1, u'sugar': 1, u'oxide': 1, u'carbon': 1, u'quikly': 1, u'dissolve': 1, u'methane': 1, u'water': 0, u'salt': 0, u'?': 0, u'survive': -1, u'fish': -1})
Counter({u'cap': 2, u'...': 1, u'rise': 1, u':': 1, u'astrology': 1, u'sun': 0, u'moon': 0, u'?': 0, u'ascendant': -1, u'be': -1, u'capricorn': -1, u',': -1, u')': -1, u'triple': -1, u'(': -1})
Counter({u'buy': 1, u'tiago': 1, u'?': 0, u'childern': -1, u'far': -

In [22]:
def make_vecs(corpus, results=False):
    vect_list = []
    results = []
    n = len(corpus)
    for i in range(n): 
        if i % 10000 == 0: print i
        f = corpus.loc[i]
        q1, q2 = f['question1'], f['question2']
        try: 
            p1 = parse(q1)
            p2 = parse(q2)
            diff1 = p1 - p2
            diff2 = p2 - p1
            vect_list.append((diff1, diff2))
            if results: results.append(f['is_duplicate'])
        except Exception as e:
            vect_list.append('')
            print(i, e)
    return fv, results

In [19]:
fv_train, target = make_vecs(train, results=True)

0
10000
20000
30000
40000


KeyboardInterrupt: 

In [ ]:
fv_test, results = make_vecs(test)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000


In [36]:
# with open('fv_train','w') as f:
#     pickle.dump(fv_train_,f)
# with open('target','w') as f:
#     pickle.dump(target,f)
with open('fv_test','w') as f:
    pickle.dump(fv_test,f)

load from pickle

In [42]:
fv_train_p = pickle.load(open('fv_train', 'rb'))
fv_test_p = pickle.load(open('fv_test', 'rb'))
target_p = pickle.load(open('target', 'rb'))

In [43]:
print(fv_train_p[10])

(Counter({u'slit': 1, u'separation': 1, u'biprism': 1, u'fresnel': 1, u'method': 1, u'find': 1}), Counter({u'laptops': 1, u'thing': 1, u'reliability': 1, u'durability': 1, u'technician': 1, u'component': 1, u'tell': 1}))
(Counter(), Counter({u'happen': 1, u'indian': 1, u'steal': 1, u'government': 1}), Counter({u'dns': 1, u'hack': 1}), Counter({u'math]23^{24}[/math': 1, u'24,23': 1, u'divide': 1, u'remainder': 1, u'[': 1, u']': 1, u'find': 1}), Counter({u'survive': 1, u'fish': 1}), Counter({u'be': 1, u'ascendant': 1, u'triple': 1, u')': 1, u'(': 1, u',': 1, u'capricorn': 1}), Counter({u'phone': 1, u'game': 1, u'video': 1, u'childern': 1, u'far': 1, u'active': 1, u'keep': 1}), Counter({u'great': 1}), Counter({u'"': 4, u'&': 1}), Counter({u'free': 1, u'internet': 1}))


In [44]:
def vectorize(vectors):
    v1, v2 = zip(*vect_list_p)
    vectorizer = DictVectorizer()
    x1 = vectorizer.fit_transform(v1)
    x2 = vectorizer.fit_transform(v2)
    X = hstack([x1,x2])
    return X

In [ ]:
X = vectorize(fv_train_p)
target = results_p

In [45]:
X.shape

(404288, 116563)

#### Validation

In [46]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
clf = MultinomialNB()
scores = cross_val_score(clf, X, target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.68 (+/- 0.00)


### Testing

In [ ]:
f = open('submission', 'rb'))
f.write('test_id,is_duplicate')
fv_test = vectorize(fv_test_p)
clf = MultinomialNB().fit(X, target)
predict_proba(fv_test_p)

In [ ]:
#this procedure will be mapped to each row
def eval_sysim(row):
    q1_txt = nlp(unicode(row['question1']))
    q2_txt = nlp(unicode(row['question2']))

    # feature: similarity measure from built-in spacy
    #word net similarity? or hamming distance of the strings?
    sy_sim = q1_txt.similarity(q2_txt)

    #feature: 
    return sy_sim
    
#unit test(s)
utest = train.loc[1]
print eval_sysim(utest)
utest = train.loc[3]
print eval_sysim(utest)
utest = train.loc[9]
print eval_sysim(utest)

here we see the pitfall in relying on the similarity measure of the two doccuments as their scope differs(invest in shares vs invest in shares of **INDIA**). To extend this baseline model we will look at parts of speech and entity recognition to help parse scope.

We can however use the similarity as the first filter to remove completely unrelated questions
